### Use case notebook: schedule CCB team meetings ###
Date: 01/05/2025

In [1]:
import os
import glob
import numpy as np
from collections import deque

# There is a warning in the timeboard library that we want to suppress here
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from pathlib import Path
import time
import timeboard as tb
import timeboard.calendars.US as US
import datetime
import holidays

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import the package
%load_ext autoreload
%autoreload 2
import cadence
from cadence.utils import FileOP
from cadence.mscheduler import Meetings
from cadence.mscheduler import cyclic_permutate
# print(f'Package version: {cadence.__version__}')

### Meeting participants and settings ###

In [2]:
# Docker drive
data_dir = os.environ.get('DATA')

# Output data to the root of the repo
output_dir = os.environ.get('HOME')

group_members_file_name = 'ccb_members_2025.xlsx'
url_base = 'https://dsets.s3.us-east-1.amazonaws.com'
url = os.path.join(url_base, group_members_file_name)

# Download file
member_file = FileOP().download_from_url(url=url, download_dir=data_dir)
tm_raw = pd.read_excel(member_file)
display(tm_raw)

Created .xlsx file.


,name,email,group
0,Andreas,andreas_werdich@hms.harvard.edu,ai
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai
2,Nathan,nathan_palmer@hms.harvard.edu,ai
3,Nidia,nidia_bucarellisanchez@hms.harvard.edu,ai
4,Alex,alex_pickering@hms.harvard.edu,cbio
5,Andrew,andrew_ghazi@hms.harvard.edu,cbio
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,cbio
7,Grey,grey_kuling@hms.harvard.edu,cbio
8,Ludwig,ludwig_geistlinger@hms.harvard.edu,cbio
9,Tram,tram_nguyen@hms.harvard.edu,cbio


### Cleanup of the spreadsheet ###
Just in case that was not done. To provide a consistent look.

In [3]:
email_col = 'email'
name_col = 'name'
group_col = 'group'

# Lets do some cleanup of the spreadsheet
# small letters for the column names
tm = tm_raw.copy()
tm.columns = [col.lower() for col in tm.columns]

# small letters for emails and group, first letter capitalize for names
tm[email_col] = tm[email_col].str.lower()
tm[group_col] = tm[group_col].str.lower()
tm[name_col] = tm[name_col].str.title()

# Sort by group and then name
tm = tm.sort_values(by=['group', 'name'], ascending=True).reset_index(drop=True)

# Mark names that are not presenting
rm_name = ['Nathan', 'Ludwig', 'Gerald', 'Tyrone']
rm_name = [nm.title() for nm in rm_name]
tm = tm.assign(presenting=True)
tm.loc[tm['name'].isin(rm_name), 'presenting'] = False

# save the name ist as a csv_file
presenter_list_file_name = f'{os.path.splitext(group_members_file_name)[0]}_list.csv'
presenter_list_file = os.path.join(data_dir, presenter_list_file_name)
tm.to_csv(presenter_list_file, index=False)
display(tm)

,name,email,group,presenting
0,Andreas,andreas_werdich@hms.harvard.edu,ai,True
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai,False
2,Nathan,nathan_palmer@hms.harvard.edu,ai,False
3,Nidia,nidia_bucarellisanchez@hms.harvard.edu,ai,True
4,Alex,alex_pickering@hms.harvard.edu,cbio,True
5,Andrew,andrew_ghazi@hms.harvard.edu,cbio,True
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,cbio,True
7,Grey,grey_kuling@hms.harvard.edu,cbio,True
8,Ludwig,ludwig_geistlinger@hms.harvard.edu,cbio,False
9,Tram,tram_nguyen@hms.harvard.edu,cbio,True


In [4]:
# Remove the names that we do not want in the schedule
presenters_df = tm.loc[tm['presenting']==True]
display(presenters_df)

,name,email,group,presenting
0,Andreas,andreas_werdich@hms.harvard.edu,ai,True
3,Nidia,nidia_bucarellisanchez@hms.harvard.edu,ai,True
4,Alex,alex_pickering@hms.harvard.edu,cbio,True
5,Andrew,andrew_ghazi@hms.harvard.edu,cbio,True
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,cbio,True
7,Grey,grey_kuling@hms.harvard.edu,cbio,True
9,Tram,tram_nguyen@hms.harvard.edu,cbio,True


### Shuffle the data and create a presenter list ###

In [5]:
# Let's do a random shuffling of the data frame
random_state = 234
presenters_df_shuffled = presenters_df.sample(frac=1, random_state=random_state)

# Get the names and the groups from the new data frame
name_list = list(presenters_df_shuffled['name'].values)
group_list = list(presenters_df_shuffled['group'].values)

# Instantiate the Meetings class with the list of names and groups from the data frame
meet = Meetings(name_list=name_list, group_list=group_list)

# We create a presenter list by merging the groups so that we have a member 
# from a different group presenting each time
name_sequence = meet.create_name_sequence()
# Rotate the sequence (start with a specific name)
rotated_sequence = cyclic_permutate(name_sequence, name='Andreas')
# updated_sequence = ['Grey', 'Andrew','Alex', 'Gerald', 'Tram', 'Andreas', 'Anthony', 'Nidia', 'Tyrone']
# updated_sequence = ['Andreas', 'Andrew','Alex', 'Nidia', 'Tram', 'Grey', 'Anthony', 'Tyrone']
updated_sequence = ['Andreas', 'Anthony', 'Nidia', 'Alex', 'Grey', 'Tram', 'Andrew']
new_presenter_list = meet.create_name_sequence(name_sequence=updated_sequence, merge_groups=False)
display(meet.name_df)

,name,group
0,Andreas,ai
1,Anthony,cbio
2,Nidia,ai
3,Alex,cbio
4,Grey,cbio
5,Tram,cbio
6,Andrew,cbio


### Create the meeting schedule ###

In [7]:
start_date = '2026-01-07'
end_date = '2026-06-10'

cal = meet.create_timeboard(start_date=start_date, 
                            end_date=end_date,
                            start_name='Andreas')

skip_date = '2026-01-14'
skip_name = 'Nathan, Andreas'
skip_comment = 'AI Workshop Discussion'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

display(cal)

,date,name,group,holiday,comment
0,2026-01-07,Andreas,ai,False,None
1,2026-01-14,"Nathan, Andreas",cbio,False,AI Workshop Discussion
2,2026-01-21,Anthony,cbio,False,None
3,2026-01-28,Nidia,ai,False,None
4,2026-02-04,Alex,cbio,False,None
5,2026-02-11,Grey,cbio,False,None
6,2026-02-18,Tram,cbio,False,None
7,2026-02-25,Andrew,cbio,False,None
8,2026-03-04,Andreas,ai,False,None
9,2026-03-11,Anthony,cbio,False,None


In [8]:
# Save the presentation schedule
save_cols = ['date', 'name', 'comment']
schedule_name = f'ccb_presentations_2026_02.csv'
schedule_file = os.path.join(output_dir, schedule_name)
cal[save_cols].to_csv(schedule_file, index=False)
display(cal[save_cols])
print(schedule_file)

,date,name,comment
0,2026-01-07,Andreas,None
1,2026-01-14,"Nathan, Andreas",AI Workshop Discussion
2,2026-01-21,Anthony,None
3,2026-01-28,Nidia,None
4,2026-02-04,Alex,None
5,2026-02-11,Grey,None
6,2026-02-18,Tram,None
7,2026-02-25,Andrew,None
8,2026-03-04,Andreas,None
9,2026-03-11,Anthony,None


/app/ccb_presentations_2026_02.csv
